### Sentiment analysis of VMware feedback data using Sentiment classification based on Naive Bayes method





In [1]:
#Importing required modules

import random
import csv
import re
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [2]:
## Reading the given dataset
FEEDBACK_DATA = pd.read_csv("Export_loop-sentiment-pos-neg-train_05112020000000.csv", sep = ",")

# Intake of eval_dfuation data and cleaning it
EVAL_DATA = pd.read_csv("sentiment-eval_df.csv",sep = ",")

In [3]:
print(FEEDBACK_DATA.head())

      label                                               text
0  Negative  No one cares about marketing slides - a techni...
1  Positive  Are all three hosts providing storage/capacity...
2  Negative  would loved to had managed to get down to the ...
3  Negative  Vending machine at work is out of Dasani water...
4  Positive  RT @VMwareEdu: Paul Maritz, CEO and President ...


In [4]:
print(EVAL_DATA.head())

   id                                               text
0   0  it's not like I haven't been talking about it ...
1   1  With a modern data center, you can cut costs a...
2   2  In today's storage landscape, there is a myria...
3   3  I hoped that it could be done via GUI.  I didn...
4   4  math final tomorrow...im getting an ulcer over it


In [5]:
#Cleaning the data in Dataframe level


In [6]:
def clean_list(data_frame): 
#def clean_list(data_frame): 
  #  'removing re-tweet, user mention, hashtags and urls in dataframe level'
   
    # remove emoticons form the dataframe if any
    data_frame['text'] = data_frame['text'].replace(r'<ed>','', regex = True)
    data_frame['text'] = data_frame['text'].replace(r'\B<U+.*>|<U+.*>\B|<U+.*>','', regex = True)
    
   
    # convert all strings to lowercase
    data_frame['text'] = data_frame['text'].str.lower()
    
    #remove user mentions
    data_frame['text'] = data_frame['text'].replace(r'^(@\w+)',"", regex=True)
    
    #remove_symbols 
    data_frame['text'] = data_frame['text'].replace(r'[^a-zA-Z0-9]', " ", regex=True)

    #remove punctuations 
    data_frame['text'] = data_frame['text'].replace(r'[[]!"#$%\'()\*+,-./:;<=>?^_`{|}]+',"", regex = True)

    #remove_URL(x):
    data_frame['text'] = data_frame['text'].replace(r'http.*$', "", regex = True)
    
    #remove words of length 1 or 2 
    data_frame['text'] = data_frame['text'].replace(r'\b[a-zA-Z]{1,2}\b','', regex=True)

    #remove numbers of length 1 or 2 and years
    data_frame['text'] = data_frame['text'].replace(r'\b[0-9]{2,4}\b','', regex=True)
    
    #remove extra spaces in the feedbacks
    data_frame['text'] = data_frame['text'].replace(r'^\s+|\s+'," ", regex=True)

    return data_frame

eval_df_clean = clean_list(EVAL_DATA)

/Users/santhoshj/.local/lib/python3.7/site-packages/pandas/core/dtypes/inference.py:213: FutureWarning: Possible nested set at position 1
  re.compile(obj)


In [7]:
eval_df = EVAL_DATA
#FEED = CLEANLIST(FEEDBACK_DATA)
feed_df = FEEDBACK_DATA

In [8]:
print(feed_df)

         label                                               text
0     Negative  No one cares about marketing slides - a techni...
1     Positive  Are all three hosts providing storage/capacity...
2     Negative  would loved to had managed to get down to the ...
3     Negative  Vending machine at work is out of Dasani water...
4     Positive  RT @VMwareEdu: Paul Maritz, CEO and President ...
...        ...                                                ...
1895  Positive  It's Friday - and that means the latest episod...
1896  Positive  Your Augmentix A+1950 appears to be nothing mo...
1897  Negative  Eating breakfast. I've attempted to do my make...
1898  Positive  vSAN 6.7 update offers enhanced health checks....
1899  Negative         oh no this is going to be on TMZ tomorrow.

[1900 rows x 2 columns]


In [9]:
## Converting the read dataset in to a list of tuples, each tuple(row) contianing the feedback text and it's label
data_set = []
for index,row in feed_df.iterrows():
    data_set.append((row['text'], row['label']))
    
eval_df_data1 = []
for index,row in eval_df.iterrows():
    eval_df_data1.append((row['text']))

In [10]:
print(data_set)

[('No one cares about marketing slides - a technical HOW TO is a lot better', 'Negative'), ("Are all three hosts providing storage/capacity? Or is one a witness? If all three have capacity drives, what'd you do with the witness? (and with them being NUCs, just how are you handling capacity and cache?)  I'm really wondering about two-node performance, as I currently have three hosts but one of them has FreeNAS virtualized and all the disk slots on that host are dedicated for that. I could add a DAS shelf or something, but I am very curious what it's like running v SAN with both capacity/cache only attached to two nodes and as far as the cluster is aware the third host is strictly only a VM host. And if that's where I'm heading, is the StarWind route going to offer something for performance and reliability with two storage nodes that VMware vSAN wouldn't be able to match with only two storage nodes?", 'Positive'), ("would loved to had managed to get down to the Camden Crawl show, but it'

In [11]:
print(eval_df_data1[:5])

[' not like haven been talking about for 6 8 years own blog ', 'with modern data center you can cut costs and improve security join houston tomorrow learn more ', ' today storage landscape there myriad choices find out why vmware managed storage your answer ', ' hoped that could done via gui didn even know about powercli but try total newb not sure succeed just can understand why such obviously demanded feature not the box thanks for your help ', 'math final tomorrow getting ulcer over ']


In [12]:
print(len(data_set))
print(len(eval_df_data1))

1900
211


### Preprocessing

In [13]:
## initialise the inbuilt STEMMER and the Lemmatizer
STEMMER = PorterSTEMMER()
WORDNET_LEMMATIZER = WordNetLemmatizer()

In [14]:
def preprocess(document, stem=True):
    'changes document to lower case, and lemmatizes/stems the remainder of the sentence'

    # change sentence to lower case
    #document = document.lower()

    # tokenize into words
    words = word_tokenize(document)

    if stem:
        words = [stemmer.stem(word) for word in words]
    else:
        words = [WORDNET_LEMMATIZER.lemmatize(word, pos='v') for word in words]

    # join words to make sentence
    document = " ".join(words)
    
    return document

In [15]:
## - Performing the preprocessing steps on all messages
train_messages = []
for (message, label) in data_set:
    words_filtered = [e for e in preprocess(message, stem=False).split() if len(e) >= 3]
    train_messages.append((words_filtered, label))
    
messages_set_eval_df = []
for (message) in eval_df_data1:
    words_filtered = [e for e in preprocess(message, stem=False).split() if len(e) >= 3]
    messages_set_eval_df.append((words_filtered, label))
    


In [16]:
print(train_messages[:5])

[(['one', 'care', 'about', 'market', 'slide', 'technical', 'HOW', 'lot', 'better'], 'Negative'), (['Are', 'all', 'three', 'host', 'provide', 'storage/capacity', 'one', 'witness', 'all', 'three', 'have', 'capacity', 'drive', 'what', 'you', 'with', 'the', 'witness', 'and', 'with', 'them', 'NUCs', 'just', 'how', 'you', 'handle', 'capacity', 'and', 'cache', 'really', 'wonder', 'about', 'two-node', 'performance', 'currently', 'have', 'three', 'host', 'but', 'one', 'them', 'have', 'FreeNAS', 'virtualized', 'and', 'all', 'the', 'disk', 'slot', 'that', 'host', 'dedicate', 'for', 'that', 'could', 'add', 'DAS', 'shelf', 'something', 'but', 'very', 'curious', 'what', 'like', 'run', 'SAN', 'with', 'both', 'capacity/cache', 'only', 'attach', 'two', 'nod', 'and', 'far', 'the', 'cluster', 'aware', 'the', 'third', 'host', 'strictly', 'only', 'host', 'And', 'that', 'where', 'head', 'the', 'StarWind', 'route', 'offer', 'something', 'for', 'performance', 'and', 'reliability', 'with', 'two', 'storage', 'n

In [17]:
print(messages_set_eval_df[:5])

[(['not', 'like', 'haven', 'talk', 'about', 'for', 'years', 'own', 'blog'], 'Negative'), (['with', 'modern', 'data', 'center', 'you', 'can', 'cut', 'cost', 'and', 'improve', 'security', 'join', 'houston', 'tomorrow', 'learn', 'more'], 'Negative'), (['today', 'storage', 'landscape', 'there', 'myriad', 'choices', 'find', 'out', 'why', 'vmware', 'manage', 'storage', 'your', 'answer'], 'Negative'), (['hop', 'that', 'could', 'via', 'gui', 'didn', 'even', 'know', 'about', 'powercli', 'but', 'try', 'total', 'newb', 'not', 'sure', 'succeed', 'just', 'can', 'understand', 'why', 'such', 'obviously', 'demand', 'feature', 'not', 'the', 'box', 'thank', 'for', 'your', 'help'], 'Negative'), (['math', 'final', 'tomorrow', 'get', 'ulcer', 'over'], 'Negative')]


### Preparing to create features

In [18]:
## - creating a single list of all words in the entire dataset for feature list creation

def get_words_in_messages(messages):
    all_words = []
    for (message, label) in messages:
      all_words.extend(message)
    return all_words

In [19]:
## - creating a final feature list using an intuitive FreqDist, to eliminate all the duplicate words
## Note : we can use the Frequency Distribution of the entire dataset to calculate Tf-Idf scores like we did earlier.

def get_word_features(wordlist):

    #print(wordlist[:10])
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [20]:
## - creating the word features for the entire dataset
word_features = get_word_features(get_words_in_messages(train_messages))
print(len(word_features))

7133


In [21]:
word_features1 = get_word_features(get_words_in_messages(messages_set_eval_df))
print(len(word_features1))

1425


### Preparing to create a train and test set

In [22]:
## - creating slicing index at 80% threshold
#sliceIndex = int((len(messages_set)*.8))

In [23]:
## - shuffle the pack to create a random and unbiased split of the dataset
#random.shuffle(train_messages)
#random.shuffle(messages_set_eval_df)

In [24]:
#train_messages, test_messages = messages_set[:sliceIndex], messages_set[sliceIndex:]
#train_messages = messages_set

In [38]:
print(len(train_messages))
print(len(messages_set_eval_df))

1900
211


### Preparing to create feature maps for train and test data

In [26]:
## creating a LazyMap of feature presence for each of the 8K+ features with respect to each of the SMS messages
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [27]:
def extract_features1(document):
    document_words = set(document)
    features = {}
    for word in word_features1:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [28]:
## - creating the feature map of train and test data

training_set = nltk.classify.apply_features(extract_features, train_messages)
testing_set = nltk.classify.apply_features(extract_features1, messages_set_eval_df)

In [29]:
print(training_set[:5])

[({'contains(one)': True, 'contains(care)': True, 'contains(about)': True, 'contains(market)': True, 'contains(slide)': True, 'contains(technical)': True, 'contains(HOW)': True, 'contains(lot)': True, 'contains(better)': True, 'contains(Are)': False, 'contains(all)': False, 'contains(three)': False, 'contains(host)': False, 'contains(provide)': False, 'contains(storage/capacity)': False, 'contains(witness)': False, 'contains(have)': False, 'contains(capacity)': False, 'contains(drive)': False, 'contains(what)': False, 'contains(you)': False, 'contains(with)': False, 'contains(the)': False, 'contains(and)': False, 'contains(them)': False, 'contains(NUCs)': False, 'contains(just)': False, 'contains(how)': False, 'contains(handle)': False, 'contains(cache)': False, 'contains(really)': False, 'contains(wonder)': False, 'contains(two-node)': False, 'contains(performance)': False, 'contains(currently)': False, 'contains(but)': False, 'contains(FreeNAS)': False, 'contains(virtualized)': False

In [30]:
print(testing_set[:5])

[({'contains(not)': True, 'contains(like)': True, 'contains(haven)': True, 'contains(talk)': True, 'contains(about)': True, 'contains(for)': True, 'contains(years)': True, 'contains(own)': True, 'contains(blog)': True, 'contains(with)': False, 'contains(modern)': False, 'contains(data)': False, 'contains(center)': False, 'contains(you)': False, 'contains(can)': False, 'contains(cut)': False, 'contains(cost)': False, 'contains(and)': False, 'contains(improve)': False, 'contains(security)': False, 'contains(join)': False, 'contains(houston)': False, 'contains(tomorrow)': False, 'contains(learn)': False, 'contains(more)': False, 'contains(today)': False, 'contains(storage)': False, 'contains(landscape)': False, 'contains(there)': False, 'contains(myriad)': False, 'contains(choices)': False, 'contains(find)': False, 'contains(out)': False, 'contains(why)': False, 'contains(vmware)': False, 'contains(manage)': False, 'contains(your)': False, 'contains(answer)': False, 'contains(hop)': False

In [31]:
print('Training set size : ', len(training_set))
print('Test set size : ', len(testing_set))

Training set size :  1900
Test set size :  211


### Training

In [32]:
## Training the classifier with NaiveBayes algorithm
Spam_Classifier = nltk.NaiveBayesClassifier.train(training_set)

### eval_dfuation

In [33]:
## - Analyzing the accuracy of the training set
print(nltk.classify.accuracy(Spam_Classifier, training_set))

0.9252631578947368


In [34]:
## Analyzing the accuracy of the test set
#print(nltk.classify.accuracy(Spam_Classifier, testing_set))

0.46445497630331756


In [35]:
print('Classification result :')
for i in range(len(eval_df_clean)):
    print(i, ':', eval_df_data['text'][i], '\n','Result:',Spam_Classifier.classify(extract_features(eval_df_clean['text'][i].split())))


Classification result :
0 :  not like haven been talking about for 6 8 years own blog  
 Result: Positive
1 : with modern data center you can cut costs and improve security join houston tomorrow learn more  
 Result: Positive
2 :  today storage landscape there myriad choices find out why vmware managed storage your answer  
 Result: Negative
3 :  hoped that could done via gui didn even know about powercli but try total newb not sure succeed just can understand why such obviously demanded feature not the box thanks for your help  
 Result: Positive
4 : math final tomorrow getting ulcer over  
 Result: Negative
5 : huh wasn expecting that show twitter time fix that  
 Result: Negative
6 : with the labs available online now took the approach not doing any world and use that time for more sessions last year  
 Result: Positive
7 :  findings were mismatched mtu settings the switch storage didn have the time figure out the exact culprit and those parts aren responsibility but scaling down re

53 :  wish you guys would get things solved with travelport asap book lot corp travel you now more difficult  
 Result: Positive
54 : wonders how some people can treat the elderly they dumb wish could with nan right now  
 Result: Negative
55 :  that awesome you were summer school teacher wouldn bore sure cold here  
 Result: Negative
56 : this what happened class while back they told sign this document allow access the online resource you will have wait 2 weeks for physical book pretty frustrating  
 Result: Positive
57 : don miss mobility featured tasks today this week cloudcred plus your chance win webcam soon  
 Result: Positive
58 :  but done trying keep current not worth the effort have dive deep the weeds every couple years  
 Result: Negative
59 :  install esxi card flash drive could boot off that and see still crashes that would least tell the vms are crashing the host its esxi right  
 Result: Negative
60 :  wish could have made  
 Result: Negative
61 :  that awesome then sho

125 : thanks for the heads the last diagram the supported deployment models list what hoping achieve however appreciate with current setup run the risk moving into the first depreciated model  
 Result: Positive
126 :  quot give lindsay hug for and thank her for the music ipod tell her love her and ill see her when she gets back quot miss her 
 Result: Negative
127 :  did that and then the won boot all and only have 1 booted all time the win10 was only for testing 
 Result: Negative
128 : yep not working like that you have shutdown freeze the vms put the host maintenance and patch the host manually via cli curious though why vcenter when you have only one host  
 Result: Positive
129 : thanks for the suggestion unfortunately doing disconnect reconnect didnt resolve the issue hoping support has resolution  
 Result: Negative
130 : sing with rain rain away  
 Result: Negative
131 : awesome wildcard cert must  
 Result: Negative
132 :  never used vgpu for our vdi did test once but since y

180 :  think your idea using the passmark template for sizing isn too bad obviously the passmark score stresses certain aspects cpu more than others integer amp performance over cache and bandwidth the actual extent the performance improvement will depend your application there way you can benchmark your vms though some kind standard which you measure their performance try running one each the types hardware with the same number vcpus per and see what kind performance difference see the application itself  
 Result: Positive
181 :  can find anyone else know who twitter alone here 
 Result: Negative
182 : agreed sounds like management are trying point fingers rather than concentrating the actual root cause typical  
 Result: Negative
183 : size your shared storage for not capacity always you almost always run out before capacity any production storage  
 Result: Positive
184 : and 10gb actually only generic number the driver with the right cpus and memory config you can actually get mor

In [36]:

with open('result5.csv', 'w+') as f:
    WRITER = csv.writer(f)
    for i in range(len(eval_df_clean)):
        WRITER.writerow((i, Spam_Classifier.classify(extract_features(eval_df_clean['text'][i].split()))))

In [1029]:
## Priting the most informative features in the classifier
print(Spam_Classifier.show_most_informative_features(30))

Most Informative Features
          contains(http) = True           Positi : Negati =     40.7 : 1.0
         contains(thank) = True           Positi : Negati =     16.1 : 1.0
         contains(sleep) = True           Negati : Positi =     10.3 : 1.0
          contains(vSAN) = True           Positi : Negati =      9.1 : 1.0
       contains(Virtual) = True           Positi : Negati =      9.0 : 1.0
          contains(fuck) = True           Negati : Positi =      8.0 : 1.0
      contains(Congrats) = True           Positi : Negati =      7.9 : 1.0
         contains(cloud) = True           Positi : Negati =      7.9 : 1.0
        contains(future) = True           Positi : Negati =      7.9 : 1.0
       contains(VMworld) = True           Positi : Negati =      7.6 : 1.0
         contains(error) = True           Negati : Positi =      7.5 : 1.0
      contains(interest) = True           Positi : Negati =      7.5 : 1.0
         contains(Check) = True           Positi : Negati =      7.3 : 1.0